# **Final Project for Numerical Analysis**

----

## *Analyzing efficacy of Gaussian Quadrature on Polynomial and Non-Polynomial Functions*

*Note: The book used for this class was "Numerical Analysis: The Mathematics of Scientific Computing, Kincaid and Cheney, 3rd edition". Any reference(s) to "the book" refer to this book.*

*Part One: Defining glaser_liu_rokhlin(), which we will refer to as GLR(), an algorithm which computes Gaussian Quadrature Formulas for function of degree n is O(n) time. 

While GLR(), based on a [paper](https://apps.dtic.mil/dtic/tr/fulltext/u2/a635869.pdf) by Andreas Glaser, Xiangtao Liu, and Vladimir Rokhlin, can work with various families of orthogonal polynomials, for our purposes we modified it to work specifically for Legendre polynomials, thereby allowing us to cut out some of the code used to generalize the algorithm "and make it work more efficiently for the Legendre polynomials.*

In [2]:
using Polynomials, SpecialPolynomials, MTH229, Plots

function newton(p, x0)
    maxsteps = 25
    dp  = derivative(p)
    while maxsteps > 0
        px, dpx = p(x0),  dp(x0) #orthogonal_polyval_derivative(p, x0)
        Δ = px/dpx
        isnan(Δ) && return (x0, dpx)
        if isinf(Δ)
            x0 += sqrt(eps())  # give a nudge from minimum or maximum
            continue
        end
        x0 -= Δ
        min(abs(Δ),abs(px)) <= sqrt(eps())/100 && return  x0, dp(x0)# orthogonal_polyval_derivative(p, x0)[2]
        maxsteps -= 1
    end
    @warn "Newton's method did not converge from $x0"
    NaN
end
function RK(t0,  x0, F, h, n)
    k0 = h*F(t0,x0)
    local x1
    for i in 1:n
        t1 = t0 + h
        k1 = h * F(t0+h,x0+k0)
        x1 = x0 + 1/2 * (k0+k1)
        t0, x0, k0 = t1, x1, k1
    end
    x1
end

function prufer(π)
    n = Polynomials.degree(π)
    Π = typeof(π)
    (θ, x) -> begin
        dom = domain(Π)
        a,b,c,d, e = SpecialPolynomials.abcde(Π)
        p =  a*x^2 + b*x  +  c
        dp = 2a*x + b
        q = d*x +  e
        dq = d
        r  = -(a*n*(n-1)  + d*n)
        dr = 0
        -inv(sqrt(r/p) + (dr*p - dp*r + 2r*q)/(2r*p) * sin(2θ)/2)
    end
end


function glaser_liu_rokhlin_gauss_nodes(π, x0=0;
                                        m=10)
    symmetry = true
    n = Polynomials.degree(π)
    F = prufer(π)
    x = float(x0)

    # step 1 to get initial might be newton or might be RK
    if symmetry && iseven(n)
        # a max at π(x0)
        x = RK(pi/2, x0, F, -(pi/2)/m, m)
    end
    
    x1, dπ1 = newton(π, x)
    rts, dπrts = [x1], [dπ1]
    N = symmetry ? div(n+1,2) : n
    for i in 2:N
        xx = RK(pi/2, rts[end], F, -pi/m, m)
        x, dπx = newton(π, xx)
        push!(rts, x)
        push!(dπrts, dπx)
    end
    if symmetry
        if iseven(n)
            rts = vcat(-reverse(rts), rts)
            dπrts = vcat(-reverse(dπrts), dπrts)
        else
            rts = vcat(-reverse(rts[2:end]), rts)
            dπrts = vcat(-reverse(dπrts[2:end]), dπrts)            
        end
    end
    # get weights 
    weights = @. (1 * 2)/(1-(rts)^2)/((Polynomials.derivative(π)(rts))^2)
    rts,  weights
end

#a1, a2 = glaser_liu_rokhlin_gauss_nodes(basis(Legendre, 6))
#@show a1, a2
#plot(a1, a2)
# x = variable()
# xs = range(-pi/2, pi/2, length = 200)
# ys =x.(xs)
# plot(xs, (ys))

glaser_liu_rokhlin_gauss_nodes (generic function with 2 methods)

### Demonstrating 7.3: Theorem 1
#### In Chapter 7.3 of the book, we see the following theorem:
*Let w be a positive weight frunction and let q be a nonzero polynomial of degree n+1 that is w-* **orthogonal** *to $\Pi_{n}$ in the sense that for any p in $\Pi_{n}$, we have</br>*  $\int\limits_a^b$ q(x)p(x)w(x)dx = 0
    *If* $x_{0}$, $x_{1}$, ... , $x_{n}$ *are the zeros of q, then the quadrature formula,* $\int\limits_a^b$ f(x)w(x)dx $\approx$ $\sum_{i=0}^{n}A_i f(x_i)$*, with coefficients given by*  $A_i$ = $\int\limits_a^b$ w(x) $\Pi_{j=0}^{n}$ $\frac{x-x_j}{x_i-x_j}$dx will be exact for all $ f ∈ \Pi_{2n+1}$ 

Here we demonstrate this theorem, the 'n's are shifted over however, so the quadrature formula will be "exact" up to but not including 2n (I put "exact" in quotes because techinically it is only as precise up to 10/20 decimal places etc.): 

In [ ]:
n = 50
#f(x) = x^5
xs, ws = glaser_liu_rokhlin_gauss_nodes(basis(Legendre, n))
for i in n:2n
    p = basis(Legendre, i)
    q = Polynomials.integrate(p, -1, 1)
    qq = sum(p(x)*w for (x,w) in zip(xs, ws))
    #@show "On iteration ", i, ", the integral of ", p, " from -1 to 1 is ", q, ", the approximated area is, ",  qq, "the approximation is accurate to ", q-qq
    @show i, q, qq, q-qq
end

#integrate(f(x), -1, 1)
# f(x) = exp(x)
# @show a = f(1)-f(-1)
# xs, ws = glaser_liu_rokhlin_gauss_nodes(basis(Legendre, 9))
# @show sum(f(x)*w for (x,w) in zip(xs, ws)) - a

### Efficacy of GLR() on Non-Polynomial Functions

GLR() on e^x. Notice how we get accuracy to 12 decimal places using only 6 points!

In [ ]:
f(x) = exp(x)
MTH229.integrate(f, -1, 1)
@show a = f(1)-f(-1)
for i in 1:12
    xs, ws = glaser_liu_rokhlin_gauss_nodes(basis(Legendre, i))
    @show i, sum(f(x)*w for (x,w) in zip(xs, ws)) - a
end

In [1]:
f(x) = cos(x)
a = f(1)-f(-1)
for i in 1:12
    xs, ws = glaser_liu_rokhlin_gauss_nodes(basis(Legendre, i))
    @show i, sum(f(x)*w for (x,w) in zip(xs, ws)) - a
end

UndefVarError: UndefVarError: basis not defined

In [ ]:
5/-4 x^-4 -5/4 - -5/4

In [5]:
f(x) = 1/ sqrt(1-x^2)
bound = 1/3+1/3+1/3.1
MTH229.integrate(f, -bound, bound)
@show a = f(bound)-f(-bound)
n = 750
for i in n:2n
    xs, ws = glaser_liu_rokhlin_gauss_nodes(basis(Legendre, i))
    @show i, sum(f(x)*w for (x,w) in zip(xs, ws)) - a
end
# #p(x) = ()
# p(x) = cos(x) + sin(x)
# #p(x) = x^3+x^2
# n = degree(p)
# xs, ws = glr(Legendre, n)
# @show  q = MTH229.integrate(p, -1, 1)
# qq = sum(p(x)*w for (x,w) in zip(xs, ws))
# @show convert(Int, q-qq)
# # for i in n:(2n-1)
# #     q = integrate(p, -1, 1)
# #     qq = sum(p(x)*w for (x,w) in zip(xs, ws))
# #     @show i, q - qq
# # end

a = f(bound) - f(-bound) = 0.0
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (750, 3.1392723602703776)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (751, 3.1392754478192324)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (752, 3.1392785271619448)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (753, 3.1392815983313294)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (754, 3.139284661359751)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (755, 3.139287716279479)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (756, 3.139290763122788)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (757, 3.139293801921578)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (758, 3.139296832707771)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (759, 3.1392998555129883)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (760, 3.1393028703686325)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (761, 3.1393058773062434)
(i, sum((f(x) * w for (x, w) = zip(xs,

(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (961, 3.139781545749832)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (962, 3.1397834274202965)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (963, 3.1397853051846556)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (964, 3.1397871790552148)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (965, 3.13978904904426)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (966, 3.139790915163673)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (967, 3.139792777425519)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (968, 3.139794635841478)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (969, 3.139796490423913)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (970, 3.139798341184391)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (971, 3.1398001881346502)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (972, 3.1398020312866652)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (973, 3.139803870651

(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1066, 3.1399598545248373)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1067, 3.1399613840788865)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1068, 3.1399629107699054)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1069, 3.139964434606126)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1070, 3.139965955595227)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1071, 3.1399674737452825)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1072, 3.139968989064337)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1073, 3.139970501560246)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1074, 3.139972011241168)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1075, 3.139973518114403)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1076, 3.1399750221881217)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1077, 3.139976523470047)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1078, 

(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1171, 3.14010620014241)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1172, 3.1401074679065735)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1173, 3.140108733509838)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1174, 3.1401099969581123)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1175, 3.140111258256628)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1176, 3.14011251741119)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1177, 3.140113774426958)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1178, 3.140115029309469)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1179, 3.1401162820642106)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1180, 3.1401175326965496)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1181, 3.1401187812118194)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1182, 3.1401200276155823)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1183, 3

(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1276, 3.1402284701020817)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1277, 3.140229537956008)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1278, 3.140230604139682)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1279, 3.1402316686564804)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1280, 3.1402327315106793)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1281, 3.1402337927063213)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1282, 3.1402348522468917)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1283, 3.1402359101362585)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1284, 3.1402369663788243)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1285, 3.1402380209779546)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1286, 3.140239073937437)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1287, 3.1402401252613927)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (12

(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1381, 3.1403321539673033)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1382, 3.140333065720939)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1383, 3.14033397615653)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1384, 3.140334885277277)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1385, 3.1403357930854705)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1386, 3.14033669958416)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1387, 3.140337604776147)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1388, 3.140338508664391)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1389, 3.140339411251725)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1390, 3.140340312540589)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1391, 3.140341212534194)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1392, 3.140342111234993)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1393, 3.140

(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1486, 3.1404211902639627)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1487, 3.140421977802335)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1488, 3.1404227642827616)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1489, 3.1404235497067043)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1490, 3.140424334076768)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1491, 3.1404251173952353)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1492, 3.1404258996642733)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1493, 3.1404266808853736)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1494, 3.140427461061186)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1495, 3.1404282401934407)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1496, 3.140429018284592)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (1497, 3.1404297953365012)
(i, sum((f(x) * w for (x, w) = zip(xs, ws))) - a) = (149

## using Plots
x = [36, 109,  181, 1700, 2314, 3955, 4390, 4546]
y = [.1111111, .06040555, .0497, .024705, .02117545, .0164348925, .01526195, 67/4546]
plot(x, y)
notice for 2m-1 terms it's exact for 2n is not
for e^x find how many to get 10^-12 acc
2 or 3 other functions like sinx etc.
try 1/sqrt(1-x^2)

In [ ]:
## using Plots
x = [36, 109,  181, 1700, 2314, 3955, 4390, 4546]
y = [.1111111, .06040555, .0497, .024705, .02117545, .0164348925, .01526195, 67/4546]
plot(x, y)
notice for 2m-1 terms it's exact for 2n is not
for e^x find how many to get 10^-12 acc
2 or 3 other functions like sinx etc.
try 1/sqrt(1-x^2)

In [ ]:
## using Plots
x = [36, 109,  181, 1700, 2314, 3955, 4390, 4546]
y = [.1111111, .06040555, .0497, .024705, .02117545, .0164348925, .01526195, 67/4546]
plot(x, y)
notice for 2m-1 terms it's exact for 2n is not
for e^x find how many to get 10^-12 acc
2 or 3 other functions like sinx etc.
try 1/sqrt(1-x^2)

In [ ]:
## using Plots
x = [36, 109,  181, 1700, 2314, 3955, 4390, 4546]
y = [.1111111, .06040555, .0497, .024705, .02117545, .0164348925, .01526195, 67/4546]
plot(x, y)
notice for 2m-1 terms it's exact for 2n is not
for e^x find how many to get 10^-12 acc
2 or 3 other functions like sinx etc.
try 1/sqrt(1-x^2)

In [ ]:
## using Plots
x = [36, 109,  181, 1700, 2314, 3955, 4390, 4546]
y = [.1111111, .06040555, .0497, .024705, .02117545, .0164348925, .01526195, 67/4546]
plot(x, y)
notice for 2m-1 terms it's exact for 2n is not
for e^x find how many to get 10^-12 acc
2 or 3 other functions like sinx etc.
try 1/sqrt(1-x^2)